In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

<img src="https://upload.wikimedia.org/wikipedia/commons/c/c7/HEIG-VD_Logo_96x29_RVB_ROUGE.png" alt="HEIG-VD Logo" width="250"/>

# Cours TAL - Laboratoire 6
# Trois méthodes de désambiguïsation lexicale

> François Burgener, Tiago Povoa Quinteiro

**Objectif**

L'objectif de ce laboratoire est d'implémenter et de comparer plusieurs méthodes de désambiguïsation lexicale (en anglais, *Word Sense Disambiguation* ou WSD).  Vous utiliserez un corpus avec plusieurs milliers de phrases, chaque phrase contenant une occurrence du mot anglais *interest* annotée avec le sens que ce mot véhicule dans la phrase respective.  Les méthodes sont les suivantes (elles seront décrites plus précisément plus bas) :

1. Algorithme de Lesk simplifié.
1. Utilisation de word2vec pour la similarité contexte vs. synset.
1. Classification supervisée utilisant des traits lexicaux, avec deux représentations :
  1. les mots en position -1, -2, ..., et +1, +2, ..., par rapport à *interest* ;
  1. apparition de mots indicateurs dans le voisinage de *interest*.

Les deux premières méthodes peuvent être qualifiées de *non supervisées*, mais en réalité elles n'utilisent pas l'apprentissage automatique.  Elles fonctionnent selon le même principe : comparer le contexte d’une occurrence de *interest* avec les définitions des sens (*synsets*) et choisir la définition la plus proche du contexte.  L’algorithme de Lesk définit la proximité comme le nombre de mots en commun, alors que word2vec la calcule comme la similarité de vecteurs.  

La troisième méthode vise à classifier les occurrence de *interest* en utilisant les sens comme des classes, et en utilisant comme traits les mots du contexte.  Vous utiliserez des méthodes d'apprentissage supervisé, en divisant les données en sous-ensembles d'apprentissage et de test.

## 0. Analyse des données

Téléchargez le corpus *interest* depuis le [site du Prof. Ted Pedersen](http://www.d.umn.edu/~tpederse/data.html).  Il se trouve en bas de cette page.  Téléchargez l'archive ZIP marquée *original format without POS tags* et extrayez le fichier `interest-original.txt`.  Téléchargez également le fichier `README.int.txt` indiqué à la ligne au-dessus. Veuillez répondre aux questions suivantes :

1. Veuillez recopier l'URL du fichier ZIP et celle du fichier `README.int.txt`.
2. Quel est le format du fichier `interest-original.txt` et comment sont annotés les sens de *interest* ?  Considère-t-on les pluriels aussi ?  Que se passe-t-il si une phrase contient plusieurs occurrences ?

1. URL :


* url_readme = 'https://www.d.umn.edu/~tpederse/Data/README.int.txt'
* url_zip = 'https://www.d.umn.edu/~tpederse/Data/interest-original.nopos.tar.gz'


2. Le format du fichier:



* numéro de ligne
* une phrase qui contient le mot interest
* il est annoté avec un underscore et un numéro, exemple: interset_6

3. D'après le fichier `README.int.txt`, quelles sont les définitions des six sens de *interest* annotés dans les données et quelles sont leurs fréquences ? Vous pouvez copier/coller l'extrait.

```
Sense 1 =  361 occurrences (15%) - readiness to give attention
Sense 2 =   11 occurrences (01%) - quality of causing attention to be given to
Sense 3 =   66 occurrences (03%) - activity, etc. that one gives attention to
Sense 4 =  178 occurrences (08%) - advantage, advancement or favor
Sense 5 =  500 occurrences (21%) - a share in a company or business
Sense 6 = 1252 occurrences (53%) - money paid for the use of money
```

Comme vu dans le README, voici les senses ci-dessus.

4. De quel dictionnaire viennent les sens précédents ? Où peut-on le consulter en ligne ?  Veuillez aligner les définitions du dictionnaire avec les six sens annotés (écrire p.ex., Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying").

```
Each sentences in the data file contains one sense-tagged occurrence
of the word "interest" (or "interests").  The sense tags correspond
the six non-idiomatic noun senses of "interest" defined in the
electronic version of the first edition of Longman's Dictionary of
Contemporary English.  The sense tags are appended to the end of the
word prior to the part-of-speech tag as shown in the following
example:

interest_6/NN```

Comme on peut le voir ci-dessus, ses définitions viennent de Longman's Dictionary of Contemporary English
    
1. [singular, uncountable] if you have an interest in something or someone, you want to know or learn more about them
2. [uncountable] a quality or feature of something that attracts your attention or makes you want to know more about it
3. [countable usually plural] an activity that you enjoy doing or a subject that you enjoy studying
4. [countable usually plural, uncountable] the things that bring advantages to someone or something
5. [countable] if you have an interest in a particular company or industry, you own shares in it
6. [uncountable] the extra money that you must pay back when you borrow money

Les définitions ci-dessus viennent de [https://www.ldoceonline.com/dictionary/interest](https://www.ldoceonline.com/dictionary/interest)

5. En consultant [WordNet en ligne](http://wordnetweb.princeton.edu/perl/webwn), trouvez les définitions des synsets  pour le **nom commun** *interest*.  Combien de synsets y a-t-il ?  Alignez les **définitions** de ces synsets avec les six sens ci-dessus (au besoin, fusionner ou ignorer des synsets).

```
Noun
S: (n) interest, involvement (a sense of concern with and curiosity about someone or something) "an interest in music" 
S: (n) sake, interest (a reason for wanting something done) "for your sake"; "died for the sake of his country"; "in the interest of safety"; "in the common interest"
S: (n) interest, interestingness (the power of attracting or holding one's attention (because it is unusual or exciting etc.)) "they said nothing of great interest"; "primary colors can add interest to a room"
S: (n) interest (a fixed charge for borrowing money; usually a percentage of the amount borrowed) "how much interest do you pay on your mortgage?"
S: (n) interest, stake ((law) a right or legal share of something; a financial involvement with something) "they have interests all over the world"; "a stake in the company's future"
S: (n) interest, interest group ((usually plural) a social group whose members control some field of activity and who have common aims) "the iron interests stepped up production"
S: (n) pastime, interest, pursuit (a diversion that occupies one's time and thoughts (usually pleasantly)) "sailing is her favorite pastime"; "his main pastime is gambling"; "he counts reading among his interests"; "they criticized the boy for his limited pursuits"

Il y a 7 synsets


1. S: (n) pastime, interest, pursuit (a diversion that occupies one's time and thoughts (usually pleasantly)) "sailing is her favorite pastime"; "his main pastime is gambling"; "he counts reading among his interests"; "they criticized the boy for his limited pursuits"

2. S: (n) interest, interestingness (the power of attracting or holding one's attention (because it is unusual or exciting etc.)) "they said nothing of great interest"; "primary colors can add interest to a room"

3. S: (n) interest, involvement (a sense of concern with and curiosity about someone or something) "an interest in music"

4. S: (n) interest, interest group ((usually plural) a social group whose members control some field of activity and who have common aims) "the iron interests stepped up production"

5. S: (n) interest, stake ((law) a right or legal share of something; a financial involvement with something) "they have interests all over the world"; "a stake in the company's future"

6. S: (n) interest (a fixed charge for borrowing money; usually a percentage of the amount borrowed) "how much interest do you pay on your mortgage?"
```





6. Définissez (manuellement, ou avec un peu de code Python) une liste nommée `senses1` avec les mots des définitions du README, en supprimant les stopwords (p.ex. les mots < 4 lettres).  Affichez la liste.

In [2]:
import nltk
from random import randrange
from nltk.corpus import stopwords

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/chadanlo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
import string
stop_words = set(stopwords.words('english')) 
stop_words2 = ["''", '""', '``', '--']

def tokenize_sentence(list_sent):
    '''
    Tokenize the sentences
    '''
    
    return list(map(lambda x : nltk.word_tokenize(x),list_sent))

def filter_sent(sent):
    '''
    Filter the sentence by removing stopword, word with one character and
    empty word
    '''
    
    tmp = [w for w in sent if not w in stop_words and not w in stop_words2 and len(w) > 1] 
    return [e for e in tmp if len(e) > 0]



In [21]:
# Veuillez répondre ici à la question et créer la variable 'senses1' (liste de 6 listes de chaînes).
senses1 = ['readiness to give attention',
           'quality of causing attention to be given to',
           'activity, etc. that one gives attention to',
           'advantage, advancement or favor',
           'a share in a company or business',
           'money paid for the use of money']


# TODO REMOVE
senses1_tokenize = tokenize_sentence(senses1)
print(senses1_tokenize)

print()

filtered_senses1 = [filter_sent(w) for w in tokenize_sentence(senses1)]
print(filtered_senses1)



[['readiness', 'to', 'give', 'attention'], ['quality', 'of', 'causing', 'attention', 'to', 'be', 'given', 'to'], ['activity', ',', 'etc', '.', 'that', 'one', 'gives', 'attention', 'to'], ['advantage', ',', 'advancement', 'or', 'favor'], ['a', 'share', 'in', 'a', 'company', 'or', 'business'], ['money', 'paid', 'for', 'the', 'use', 'of', 'money']]

[['readiness', 'give', 'attention'], ['quality', 'causing', 'attention', 'given'], ['activity', 'etc', 'one', 'gives', 'attention'], ['advantage', 'advancement', 'favor'], ['share', 'company', 'business'], ['money', 'paid', 'use', 'money']]


In [22]:
filtered_senses1 = [filter_sent(w) for w in senses1_tokenize]
print(filtered_senses1)

[['readiness', 'give', 'attention'], ['quality', 'causing', 'attention', 'given'], ['activity', 'etc', 'one', 'gives', 'attention'], ['advantage', 'advancement', 'favor'], ['share', 'company', 'business'], ['money', 'paid', 'use', 'money']]


7. En combinant les définitions obtenues aux points (3) et (4) ci-dessus, construisez une liste nommée `senses2` avec pour chacun des sens de *interest* une liste de **mots-clés** qui le caractérise.  Vous pouvez concaténer les définitions, puis écrire des instructions en Python pour extraire les mots (uniques).  Respectez l'ordre des sens données par `README`, et à la fin affichez `senses2`.

In [23]:
# Veuillez répondre ici à la question et créer la variable 'senses2' (liste de 6 listes de chaînes).
senses2 = [
    'if you have an interest in something or someone, you want to know or learn more about them',
    'a quality or feature of something that attracts your attention or makes you want to know more about it',
    'an activity that you enjoy doing or a subject that you enjoy studying',
    'the things that bring advantages to someone or something',
    'if you have an interest in a particular company or industry, you own shares in it',
    'the extra money that you must pay back when you borrow money'
]

senses3 = [
    "pursuit (a diversion that occupies one's time and thoughts (usually pleasantly)) sailing is her favorite pastime; his main pastime is gambling; he counts reading among his interests\"; \"they criticized the boy for his limited pursuits",
    "interestingness (the power of attracting or holding one's attention (because it is unusual or exciting etc.)) they said nothing of great interest; primary colors can add interest to a room",
    "involvement (a sense of concern with and curiosity about someone or something) an interest in music",
    "group ((usually plural) a social group whose members control some field of activity and who have common aims) the iron interests stepped up production",
    "stake ((law) a right or legal share of something; a financial involvement with something) they have interests all over the world; a stake in the company's future",
    "a fixed charge for borrowing money; usually a percentage of the amount borrowed) how much interest do you pay on your mortgage?",
]

print("senses2: ", senses2, '\n')

# Filtering the sentences in senses
filtered_senses2 = [filter_sent(w) for w in tokenize_sentence(senses2)]
filtered_senses3 = [filter_sent(w) for w in tokenize_sentence(senses3)]

# For the demo here, we only show with one sentence so it is more readable
print()
print("filtered_senses 1: ", filtered_senses1[1])
print("filtered_senses 2: ", filtered_senses2[1])
print("filtered_senses 3: ", filtered_senses3[1])

print()
print("Display all the merged senses alltogether: --------------")
print()
f_senses = zip(filtered_senses1, filtered_senses2, filtered_senses3)
f_senses = list(map(lambda tup: list(set(tup[0] + tup[1] + tup[2])), f_senses))

print("filtered_senses (merged): ", f_senses[1])
print(len(f_senses))

senses2:  ['if you have an interest in something or someone, you want to know or learn more about them', 'a quality or feature of something that attracts your attention or makes you want to know more about it', 'an activity that you enjoy doing or a subject that you enjoy studying', 'the things that bring advantages to someone or something', 'if you have an interest in a particular company or industry, you own shares in it', 'the extra money that you must pay back when you borrow money'] 


filtered_senses 1:  ['quality', 'causing', 'attention', 'given']
filtered_senses 2:  ['quality', 'feature', 'something', 'attracts', 'attention', 'makes', 'want', 'know']
filtered_senses 3:  ['interestingness', 'power', 'attracting', 'holding', 'one', "'s", 'attention', 'unusual', 'exciting', 'etc', 'said', 'nothing', 'great', 'interest', 'primary', 'colors', 'add', 'interest', 'room']

Display all the merged senses alltogether: --------------

filtered_senses (merged):  ['interest', 'attracting', '

8. Chargez les données depuis `interest-original.txt` dans une liste appelée `sentences` qui contient pour chaque phrase la liste des mots (sans les séparateurs *$$* et *===...*).  Les phrases sont-elles déjà tokenisées ?  Sinon, faites-le.  À ce stade, ne modifiez pas encore les occurrences annotées *interest(s)\_X*.  Comptez le nombre total de phrases et affichez-en quatre comme indiqué.

In [24]:
# Veuillez répondre ici à la question.
def read_filename(filename):
    sentences = []

    with open(filename, "r", encoding="utf-8") as fd:
        lines = fd.readlines()
        
        for l in lines:
            if not l.startswith('$$'):
                sentences.append(l.replace('==', ''))
                
    return sentences

sentences = read_filename('interest-original.txt')

print("Il y a {} phrases.\nEn voici 3 au hasard :".format(len(sentences)))
print(sentences[151:154])

sentences_tokenized = [filter_sent(w) for w in tokenize_sentence(sentences)]
print()
print(sentences_tokenized[151:154])

Il y a 2368 phrases.
En voici 3 au hasard :
["  investor interest_1  in  stock funds  ``  has n't  stalled at  all  , ''  mr. hines  maintains . \n", "''   it  is in  the western interest_4  to see  mr. gorbachev  succeed . \n", "  revco  insists that  the proposal  is simply  an  ``  expression  of  interest_1  , '' because under  chapter 11 revco  has ``  exclusivity rights  '' until  feb. 28  . \n"]

[['investor', 'interest_1', 'stock', 'funds', "n't", 'stalled', 'mr.', 'hines', 'maintains'], ['western', 'interest_4', 'see', 'mr.', 'gorbachev', 'succeed'], ['revco', 'insists', 'proposal', 'simply', 'expression', 'interest_1', 'chapter', '11', 'revco', 'exclusivity', 'rights', 'feb.', '28']]


## 1. Algorithme de Lesk simplifié

Définissez une fonction `wsd_lesk(senses, sentence)` qui prend deux arguments : une liste de listes de mots-clés par sens (comme `senses1` et `senses2` ci-dessus) et une phrase avec une occurrence annotée de *interest* ou *interests*, et qui retourne le numéro du sens le plus probable selon l'algorithme de Lesk.  On rappelle que cet algorithme choisit le sens qui a le maximum de mots en commun avec le voisinage de *interest*.  Vous pouvez choisir vous-même la taille de ce voisinage (*window_size*).  En cas d'égalité entre deux sens, tirer la réponse au sort.

*Note : il ne serait pas correct de choisir le sens le plus fréquent en cas d'égalité, car cela suppose qu'on a utilisé les données de test pour calculer ces fréquences, donc pour s'entraîner, ce qui est incorrect !*

In [25]:
# Veuillez répondre ici à la question.
import random

def util_max(dic):
    """
    take a frequency dictionnary
    returns the key with the most frequency
    in case of equality, returns one at random.
    """
    list_max = []
    maxVal = 0
    for key, value in dic.items():
        if value > maxVal:
            maxVal = value
            list_max = []
            list_max.append(key)
        elif value == maxVal:
            list_max.append(key)
            
    return list_max[0] if len(list_max) == 1 else random.choice(list_max) 

def wsd_lesk(senses, sentence):
    """
    identifies a probable sense for a sentence
    from frequency between the sense and the sentence
    """
    

    window_size = 5
    interest_idx = [sentence.index(w) for w in sentence if w.startswith('interest_') or w.startswith('interests_')][0]   
    start = (interest_idx - window_size) if (interest_idx - window_size) > 0 else 0
    end = (interest_idx + window_size) + 1 if (interest_idx + window_size) < len(sentence) else len(sentence)
    
    freq = {}
    sense_idx = 1
    
    
    
    for s in senses:
        common = set(sentence[start:end]).intersection(s)
        freq[sense_idx] = len(common)
        sense_idx += 1

    return util_max(freq)

a = wsd_lesk(f_senses, sentences_tokenized[104])
print(a)

2


Définissez maintenant une fonction `evaluate_wsd(fct_name, senses, sentences)` qui prend en paramètre le nom de la fonction de similarité (pour commencer : `wsd_lesk`) ainsi que la liste des sens et la liste de phrases, et retourne le score total de la méthode (pourcentage du nombre de réponses correctes par phrase) en vérifiant pour chaque phrase si le sens trouvé est identique au sens annoté.  Améliorez ensuite la méthode pour afficher le taux de correction par sens.

In [26]:
# Veuillez répondre ici à la question.
from functools import reduce

def evaluate_wsd(fct_name, senses, sentences):
    """
    Evaluates the precision of the method fct_name
    """
    _pairs_list = []
    
    for sent in sentences:
        idx = fct_name(senses, sent)
        word = [w for w in sent if w.startswith('interest_') or w.startswith('interests_')][0]
        correct_sense = int(word[-1:])
        _pairs_list.append((correct_sense, idx))
        
    res = reduce(lambda acc, pair: acc + (1 if pair[0] == pair[1] else 0), _pairs_list, 0)
    
    return res/len(_pairs_list)
    
    
print("Sense_1 score : ",evaluate_wsd(wsd_lesk, filtered_senses1, sentences_tokenized))
print("Sense_2 score : ",evaluate_wsd(wsd_lesk, filtered_senses2, sentences_tokenized))
print("Sense_3 score : ",evaluate_wsd(wsd_lesk, filtered_senses3, sentences_tokenized))

print("All senses merge score : ",evaluate_wsd(wsd_lesk, f_senses, sentences_tokenized))

Sense_1 score :  0.21452702702702703
Sense_2 score :  0.22086148648648649
Sense_3 score :  0.22212837837837837
All senses merge score :  0.22550675675675674


En fixant au mieux la taille de la fenêtre autour de *interest*, quel est le meilleur taux de correction de la méthode de Lesk simplifiée ?  Quelle liste de sens conduit à de meilleurs scores, `senses1` ou `senses2` ?

*Note : optimiser la taille de la fenêtre sur les données de test serait incorrect !*

#### Veuillez répondre ici à la question.

Ici, on constate que sense2 est meilleur que sense1.

évidemment, la version fusionnée est supérieur vu qu'elle comporte tous les mots de chaque sense.

## 2. Utilisation de word2vec pour la similarité contexte vs. synset

En réutilisant une partie du code de `wsd_lesk`, définissez maintenant une fonction `wsd_word2vec(senses, sentence)` qui choisit le sens en utilisant la similarité **word2vec**.  On vous encourage à chercher dans la [documentation des KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) comment calculer directement la similarité entre deux listes de mots.

Comme `wsd_lesk`, la nouvelle fonction `wsd_word2vec` prend en argument une liste de listes de mots-clés par sens (comme `senses1` et `senses2` ci-dessus), et une phrase avec une occurrence annotée de *interest* ou *interests*.  La fonction retourne le numéro du sens le plus probable selon la similarité word2vec entre les mots du sens et ceux du voisinage de *interest*. Vous pouvez choisir la taille de ce voisinage (`window_size`).  En cas d'égalité, tirer le sens au sort.

In [27]:
import gensim.downloader as api

w2v_model = api.load("word2vec-google-news-300")
word_vectors = w2v_model.wv

/home/chadanlo/anaconda3/envs/tal/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


In [28]:
# import gensim
# from gensim.models import KeyedVectors

# path_to_model = "../../../gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz" # à adapter
# wv_from_bin = gensim.models.KeyedVectors.load_word2vec_format(path_to_model, binary=True)  # C bin format

In [29]:
# Veuillez répondre ici à la question.
def wsd_word2vec(senses, sentence):
    
    window_size = 2
    interest_idx = [sentence.index(w) for w in sentence if w.startswith('interest_') or w.startswith('interests_')][0]   
    start = (interest_idx - window_size) if (interest_idx - window_size) > 0 else 0
    end = (interest_idx + window_size) + 1 if (interest_idx + window_size) < len(sentence) else len(sentence)
    
    # We filter out the words NOT in vocabulary.
    # Otherwise n_similarity throws an error
    sublist_sentence = list(filter(lambda x: x in w2v_model.vocab, sentence[start:end]))
        
    min_sim = 1.0
    idx_senses = 0
    for index, sense in enumerate(senses):
        # We filter out the words NOT in vocabulary.
        # Otherwise n_similarity throws an error
        _sense = list(filter(lambda x: x in w2v_model.vocab, sense))
        
        if sublist_sentence and _sense:
            try:
                sim = word_vectors.n_similarity(_sense, sublist_sentence)
                if (sim < min_sim):
                    min_sim = sim
                    idx_senses = index
            except Exception as e:
                print('Error in wsd_word2vec', e)
            
    return idx_senses + 1
            
        
wsd_word2vec(filtered_senses1,sentences_tokenized[10])
    

4

Appliquez maintenant la même méthode `evaluate_wsd` avec la fonction `wsd_word2vec` (en cherchant une bonne valeur de la taille de la fenêtre) et affichez les scores (globaux et par sens) pour la similarité word2vec.  Comment se comparent-ils avec les précédents ?

In [30]:
# Veuillez répondre ici à la question.
    
print("Sense_1 score : ",evaluate_wsd(wsd_word2vec, filtered_senses1, sentences_tokenized))
print("Sense_2 score : ",evaluate_wsd(wsd_word2vec, filtered_senses2, sentences_tokenized))
print("Sense_3 score : ",evaluate_wsd(wsd_word2vec, filtered_senses3, sentences_tokenized))

print("All senses merge score : ",evaluate_wsd(wsd_word2vec, f_senses, sentences_tokenized))

Sense_1 score :  0.0933277027027027
Sense_2 score :  0.0472972972972973
Sense_3 score :  0.05152027027027027
All senses merge score :  0.04222972972972973


### Window_size 50

* Sense_1 score :  0.078125
* Sense_2 score :  0.033361486486486486
* Sense_3 score :  0.03800675675675676
* All senses merge score :  0.024070945945945946

### Window_size 10

* Sense_1 score :  0.08023648648648649
* Sense_2 score :  0.03589527027027027
* Sense_3 score :  0.04054054054054054
* All senses merge score :  0.027449324324324325

### Window_size 5

* Sense_1 score :  0.08403716216216216
* Sense_2 score :  0.04265202702702703
* Sense_3 score :  0.04307432432432432
* All senses merge score :  0.03505067567567568

### Window_size 2

* Sense_1 score :  0.09797297297297297
* Sense_2 score :  0.052787162162162164
* Sense_3 score :  0.05405405405405406
* All senses merge score :  0.04983108108108108

### Conclusion

Ces scores ne sont vraiment pas terribles. On arrive au maximum à 9.7% avec une taille de fenere de 2. 

La méthode de similarité de Word2Vec ne permet pas la désambiguïsation.

## 3. Classification supervisée avec des traits lexicaux
Dans cette partie du labo, vous entraînerez des classifieurs pour prédire le sens d'une occurrence dans une phrase.  Le principal défi sera de transformer chaque phrase en un ensemble de traits, pour créer les données en vue des expériences de classification.

Vous utiliserez le classifieur `NaiveBayesClassifier` fourni par NLTK.  Le mode d'emploi se trouve dans le [Chapitre 6, sections 1.1-1.3](https://www.nltk.org/book/ch06.html) du livre NLTK.  Consultez-le attentivement pour trouver comment formater les données.  (Il existe de nombreux autres classifieurs supervisés, par exemple dans la boîte à outils `scikit-learn`.)

De plus, vous devrez séparer les 2368 occurrences en ensembles d'entraînement et de test.

### 3.A. Traits lexicaux positionnels

Dans cette première représentation des traits, vous les coderez comme `mot-2`, `mot-1`, `mot+1`, `mot+2`, etc. (fenêtre de taille `2*window_size` autour de *interest*) et vous leur donnerez les valeurs des mots observés aux emplacements respectifs, ou alors `NONE` si la fenêtre dépasse la limite de la phrase.  Vous ajouterez un trait qui est le mot *interest* lui-même, qui peut être au singulier ou au pluriel.  Pour chaque occurrence de *interest*, vous devez donc générer une représentation formelle avec un dictionnaire Python puis un entier :
```
[{'word-1': 'in', 'word+1': 'rates', 'word-2': 'declines', 'word+2': 'NONE', 'word0': 'interest'}, 6]
```
où l'entier est le numéro du sens (ici, 6).  Cette valeur servira à l'entraînement, puis elle sera cachée à l'évaluation, et la prédiction du système sera comparée à elle pour dire si elle est correcte ou non.  Vous regrouperez toutes ces entrées dans une liste totale de 2368 éléments appelée `items_with_features_A`.

En partant de la liste des phrases appelée `sentences`(préparée plus haut), veuillez générer ici cette liste, en vous aidant si nécessaire du livre NLTK.

In [31]:
def find_idx(sentence):
    return [sentence.index(w) for w in sentence if w.startswith('interest_') or w.startswith('interests_')][0]

def prepare_lexical_traits(sentence, window_size):
    dico = {}
    correct_sense = 0
    
    interest_idx = find_idx(sentence)
    start = (interest_idx - window_size) if (interest_idx - window_size) > 0 else 0
    end = (interest_idx + window_size) + 1 if (interest_idx + window_size) < len(sentence) else len(sentence)
    
    window_sent = sentence[start:end]
    _idx = find_idx(window_sent)
    
    for index, word in enumerate(window_sent):
        if index == _idx:
            splited = word.split('_')
            dico['word0'] = splited[0]
            correct_sense = int(splited[1])
        else:
            dico['word{}{}'.format('+' if index - _idx > 0 else '', index - _idx)] = word
    
    for i in range(-window_size, -_idx):
        dico['word{}'.format(i)] = 'NONE'

    for i in range(len(window_sent), (_idx + window_size + 1)):
        dico['word+{}'.format(i -_idx)] = 'NONE'
        
    return [dico, correct_sense]

In [46]:
WINDOW_SIZE = 3

items_with_features_A = [ prepare_lexical_traits(s, WINDOW_SIZE) for s in sentences_tokenized]

In [47]:
# Veuillez répondre ici à la question.

print(len(items_with_features_A))
for iwf in items_with_features_A[151:154]:
    print(iwf)
    print()

2368
[{'word-1': 'investor', 'word0': 'interest', 'word+1': 'stock', 'word+2': 'funds', 'word+3': "n't", 'word-3': 'NONE', 'word-2': 'NONE'}, 1]

[{'word-1': 'western', 'word0': 'interest', 'word+1': 'see', 'word+2': 'mr.', 'word+3': 'gorbachev', 'word-3': 'NONE', 'word-2': 'NONE'}, 4]

[{'word-3': 'proposal', 'word-2': 'simply', 'word-1': 'expression', 'word0': 'interest', 'word+1': 'chapter', 'word+2': '11', 'word+3': 'revco'}, 1]



On souhaite maintenant entraîner un classifieur sur une partie des données, et le tester sur une autre.  Typiquement, on peut garder 80% des données pour l'entraînement et utiliser les 20% restants pour l'évaluation.  Naturellement, on fait cette division séparément pour chaque sens, pour que les deux ensembles contiennent les mêmes proportions de sens que l'ensemble de départ.  (On parle de "*stratified split*").  Nous ferons cette division aléatoirement, une seule fois, mais typiquement on la fait plusieurs fois et on fait la moyenne des scores obtenus ("*cross-validation*").  Il y a des packages qui font cela automatiquement (p.ex. `scikit-learn`).

Veuillez maintenant deux sous-ensembles de `items_with_features_A` selon ces indications, appelés `iwf_A_train` et `iwf_A_test`.


In [48]:
from random import shuffle

In [56]:
shuffle(items_with_features_A)

SIZE = int(len(items_with_features_A) / 5)

iwf_A_train = items_with_features_A[SIZE:]
iwf_A_test  = items_with_features_A[:SIZE]
# Veuillez répondre ici à la question.

print(len(iwf_A_train), ' ', len(iwf_A_test))
print(iwf_A_test[:2], iwf_A_test[-2:])

1895   473
[[{'word-2': 'stimulate', 'word-1': 'additional', 'word0': 'interest', 'word+1': 'among', 'word+2': 'thrifts', 'word+3': 'said', 'word-3': 'NONE'}, 1], [{'word-3': 'frankfurt', 'word-2': 'authorities', 'word-1': 'move', 'word0': 'interest', 'word+1': 'rates', 'word+2': 'NONE', 'word+3': 'NONE'}, 6]] [[{'word-3': 'based', 'word-2': 'outlook', 'word-1': 'lower', 'word0': 'interest', 'word+1': 'rates', 'word+2': 'according', 'word+3': 'japanese'}, 6], [{'word-2': 'occidental', 'word-1': 'petroleum', 'word0': 'interests', 'word+1': 'oil', 'word+2': 'gas', 'word+3': 'pipelines', 'word-3': 'NONE'}, 5]]


Veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_A_train` et la tester sur `iwf_A_train` (voir la documentation NLTK).  En expérimentant avec différentes largeurs de fenêtres, quel est le meilleur score global que vous obtenez, et comment se compare-t-il avec les précédents ?  Quels sont les traits les plus informatifs, et pouvez-vous expliquer cet affichage ?

*Note : vous pouvez choisir de générer plusieurs fois aléatoirement les deux sous-ensembles, et voir comment les scores varient.*

*Note 2 : il serait possible de diviser les données en 3, et choisir la meilleur taille de la fenêtre sur un ensemble de développement, différent de celui de test final.*

In [161]:
from nltk.classify import naivebayes 
# Veuillez répondre ici à la question.

classifier = nltk.NaiveBayesClassifier.train(iwf_A_train)
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, iwf_A_test))*100)


Classifier accuracy percent: 86.4693446088795


In [163]:
classifier.show_most_informative_features(20)

Most Informative Features
                   word0 = 'interests'         3 : 1      =     63.4 : 1.0
                  word-2 = 'NONE'              6 : 3      =     26.4 : 1.0
                  word-3 = 'NONE'              6 : 3      =     22.5 : 1.0
                  word+3 = 'NONE'              6 : 2      =     21.5 : 1.0
                  word+2 = 'NONE'              6 : 2      =     15.2 : 1.0
                  word-2 = 'foreign'           6 : 5      =     12.1 : 1.0
                  word+1 = 'new'               5 : 6      =     11.5 : 1.0
                  word-1 = 'NONE'              6 : 4      =     10.9 : 1.0
                  word+1 = 'rose'              5 : 6      =     10.3 : 1.0
                  word-3 = 'u.s.'              6 : 4      =      8.3 : 1.0
                  word-1 = '50'                5 : 6      =      7.8 : 1.0
                  word+3 = 'company'           5 : 6      =      7.4 : 1.0
                  word-3 = 'co.'               5 : 6      =      7.3 : 1.0

Le score est largement meilleur, dans notre cas, qu'avec word2vec et un algo de Lesk simplifié.

Les traits les plus informatifs sont: 
* savoir si interest est au pluriel
* S'il est plutôt au début ou à la fin
* l'apparition de certains mots particuliers sont très informatifs pour certaines classes:
    * foreign, new, rose, u.s., 50, company. co., pursue, strong, board, holding, corp.
    
Cet affichage nous fournit des positifs contre négatifs. C'est des probabilités de vraissemblance.



On souhaite également obtenir les scores pour chaque sens.  Pour ce faire, il faut demander les prédictions une par une au classifieur (cherchez dans le [livre NLTK](https://www.nltk.org/book/ch06.html) comment), et comptabiliser les prédictions correctes pour chaque sens.  Suggestion : inspirez-vous de `evaluate_wsd`, mais appliquez-là seulement aux données `iwf_A_test`.  Veuillez écrire une fonction nommée ainsi : `evaluate_wsd_supervised(classifier, items_with_features)`.

In [164]:
# Veuillez répondre ici à la question.
def evaluate_wsd_supervised(classifier, items_with_features):
    _tmp_truth = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}
    _tmp_found = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}
    
    for item in items_with_features:
        found = classifier.classify(item[0])
        truth = item[1]
        _tmp_truth[truth] += 1
        _tmp_found[truth] += 1 if truth == found else 0      
    
    result = []
    for k, v in _tmp_truth.items():
        result.append('Sense {} score: {} over a size of {}'.format(k, ( _tmp_found[k] / v), v))
        
    return result

In [208]:
res = evaluate_wsd_supervised(classifier, iwf_A_test)

for l in res:
    print(l)

Sense 1 score: 0.8051948051948052 over a size of 77
Sense 2 score: 0.0 over a size of 1
Sense 3 score: 0.6666666666666666 over a size of 15
Sense 4 score: 0.6666666666666666 over a size of 33
Sense 5 score: 0.7623762376237624 over a size of 101
Sense 6 score: 0.967479674796748 over a size of 246


Il n'y avait qu'un seul élément de Sense 2. Il s'est trompé donc précision de 0%. Dommage

### 3.B. Présence de mots indicateurs

Une deuxième façon d'encoder les traits lexicaux est de constituer un vocabulaire avec les mots qui apparaissent dans le voisinage de *interest* et de définir ces mots comme traits.  Par conséquent, pour chaque occurrence de *interest*, vous allez extraire la valeur de ces traits sous la forme :
```
[{('rate' : True), ('in' : False), ...}, 1]
```
où *'rate'*, *'in'* sont les mots du vocabulaire, True/False indiquent leur présence/absence autour de l'occurrence de *interest* qui est décrite, et le dernier nombre est le sens, entre 1 et 6.

Pour commencer, en partant de `sentences` et en fixant la taille de la fenêtre, veuillez constituer la liste de tous les mots observés autour de tous les voisinages de toutes les occurrences de *interest*.

In [58]:
WINDOW_SIZE = 3

__items = [ prepare_lexical_traits(s, WINDOW_SIZE) for s in sentences_tokenized]    

In [69]:
word_list = []
# Veuillez répondre ici à la question.

for sent in __items:
    for _, word in sent[0].items():
        if word != 'NONE':
            word_list.append(word)

print(len(word_list))
print(word_list[:50])

14803
['managers', 'expect', 'declines', 'interest', 'rates', 'thought', 'indicate', 'declining', 'interest', 'rates', 'permit', 'portfolio', 'recent', 'rises', 'short-term', 'interest', 'rates', 'co.', 'holds', '83.4', 'interest', 'energy-services', 'company', 'elected', 'state-owned', 'holding', 'company', 'interests', 'mechanical', 'engineering', 'industry', 'unreasonable', 'refunded', 'plus', 'interest', 'judge', 'curry', 'set', 'interest', 'rate', 'refund', 'property', 'country', "'s", 'interest', 'prompted', 'improvements', 'made', 'significant', 'reduction']


En utilisant un objet `nltk.FreqDist`, veuillez sélectioner les 500 mots les plus fréquents (vous pourrez aussi faire varier ce nombre), dans une liste appelée `vocabulary`.  À votre avis, est-ce une bonne idée d'enlever les *stopwords* de cette liste pour construire les traits ?

In [72]:
# Veuillez répondre ici à la question.
fdist = nltk.FreqDist(word_list)

vocabulary = list(map(lambda p: p[0], fdist.most_common(500)))

print(vocabulary[:50])

['interest', 'rates', 'interests', "'s", 'rate', 'payments', 'company', 'u.s.', 'lower', "n't", 'said', 'million', 'bonds', 'would', 'high', 'short', 'higher', 'foreign', 'income', 'annual', 'minority', 'also', 'debt', 'pay', 'general', 'short-term', 'buying', '--', 'mr.', 'market', 'federal', 'guide', 'due', 'bank', 'new', 'pursue', 'reserve', 'current', 'says', 'net', 'year', 'plus', 'best', 'accrued', 'public', 'principal', 'increase', 'investor', 'inflation', 'equity']


#### Stopwords

Alors probablement oui. Les stopwords transportent peu de valeur.

Après, peut-être que voir apparaître certains symboles tel que '%' peut avoir un impact à aider à identifier certains sens de interest

---

Veuillez maintenant créer l'ensemble total de données formatées, en convertissant chaque phrase contenant une occurrence de *interest* à un dictionnaire de traits/valeurs (suivi du numéro du sens), comme exemplifié au début de cette section 3B.  Cet ensemble sera appelé `items_with_features_B`.

In [198]:
all_voc_dic = {}

for word in word_list:
    all_voc_dic[word] = False

In [199]:
_tmp_list = []
_tmp = {}

for sent in __items:
    _tmp.update(all_voc_dic)

    for _, word in sent[0].items():
        if word != 'NONE':
            _tmp[word] = True
            sense = sent[1]
            
    _tmp_list.append([_tmp, sense])
    _tmp = {}


In [200]:
items_with_features_B = _tmp_list
# Veuillez répondre ici à la question.

print(len(items_with_features_B))

2368


Comme dans la section 3A, veuillez créer maintenant deux sous-ensembles de `items_with_features_B` appelés `iwf_B_train` (80% des items) et `iwf_B_test` (20% des items), avec une sélection aléatoire mais stratifiée.

In [202]:
shuffle(items_with_features_B)

SIZE = int(len(items_with_features_B) / 5)

iwf_B_train = items_with_features_B[SIZE:]
iwf_B_test  = items_with_features_B[:SIZE]
# Veuillez répondre ici à la question.

print(len(iwf_B_train), ' ', len(iwf_B_test))

1895   473


Comme pour la section 3A, veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_B_train` et la tester sur `iwf_B_train`.  Veuillez tester ce classifieur globalement, puis pour chacun des six sens (classes) avec la méthode `evaluate_wsd_supervised` du 3A.

En expérimentant avec différentes largeurs de fenêtres et tailles du vocabulaire, quel est le meilleur score que vous obtenez, et comment se compare-t-il avec les précédents ?  Quels sont les traits les plus informatifs ?

In [203]:
from nltk.classify import naivebayes 
# Veuillez répondre ici à la question.

classifier_2 = nltk.NaiveBayesClassifier.train(iwf_B_train)
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier_2, iwf_B_test))*100)

Classifier accuracy percent: 84.77801268498943


In [206]:
classifier_2.show_most_informative_features(10)

Most Informative Features
                  pursue = True                3 : 5      =    127.5 : 1.0
              particular = True                2 : 6      =     92.3 : 1.0
                  little = True                2 : 6      =     92.3 : 1.0
              especially = True                2 : 6      =     92.3 : 1.0
                    came = True                2 : 6      =     92.3 : 1.0
                national = True                2 : 6      =     92.3 : 1.0
                    deal = True                2 : 6      =     92.3 : 1.0
                   rates = True                6 : 1      =     90.2 : 1.0
                interest = False               3 : 1      =     85.7 : 1.0
               interests = True                3 : 1      =     85.7 : 1.0


Il est très équivalent au système précédent, mais nous renseigne de manière différente sur des caractéristiques intéressantes. Aussi, il n'utilise plus l'aspect positionnel

Les traits les plus informatifs sont la présence de certaints mots spécifiques à une catègorie.

On voit que pursue, particular, little,... sont très forts. 

In [166]:
# Veuillez recopier ici en conclusion les scores par sens des quatre 
# expériences, pour pouvoir les comparer d'un coup d'oeil.


Par rapport aux autres méhotdes, le classificateur est largement supérieur.

* 1 Lesk simplifié: 22%
* 2 Word2Vec: 9.7%
* 3a) 86%
* 3b) 84%

## Fin du laboratoire

Merci de nettoyer votre feuille, exécuter une dernière fois toutes les instructions, et sauvegarder le résultat.  
Comprimez la feuille dans un fichier `.zip` et soumettez-le sur Cyberlearn.